In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

#인지 정상(CN), 경도 인지 장애(MCI) 및 치매(DEM)
file_path = "1.Training/원천데이터/1.걸음걸이/train_activity.csv"
Walk = pd.read_csv(file_path)
file_path = "1.Training/라벨링데이터/1.걸음걸이/training_label.csv"
Walk_label = pd.read_csv(file_path)
Walk_label.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
Walk = pd.merge(Walk, Walk_label[['EMAIL', 'DIAG_NM']], on='EMAIL', how='left')
Walk['EMAIL'] = Walk['EMAIL'].str.extract(r'(\d{3})')
Walk.rename(columns={
    'activity_average_met': '하루간 평균 MET',
    'activity_cal_active': '하루간 활동 칼로리',
    'activity_cal_total': '하루간 총 사용 칼로리',
    'activity_class_5min': '하루간 5분당 활동 로그',
    'activity_daily_movement': '매일 움직인 거리',
    'activity_day_end': '활동 종료 시간',
    'activity_day_start': '활동 시작 시간',
    'activity_high': '고강도 활동 시간',
    'activity_inactive': '비활동 시간',
    'activity_inactivity_alerts': '비활동 알람 횟수',
    'activity_low': '저강도 활동 시간',
    'activity_medium': '중강도 활동 시간',
    'activity_met_1min': '하루간 1분 당 MET 로그',
    'activity_met_min_high': '하루간 고강도 활동 MET',
    'activity_met_min_inactive': '하루간 비활동 MET',
    'activity_met_min_low': '하루간 저강도 활동 MET',
    'activity_met_min_medium': '하루간 중강도 활동 MET',
    'activity_non_wear': '미착용 시간',
    'activity_rest': '휴식 시간',
    'activity_score': '활동 점수',
    'activity_score_meet_daily_targets': '활동 목표달성 점수',
    'activity_score_move_every_hour': '매 시간 당 활동유지 점수',
    'activity_score_recovery_time': '회복시간 점수',
    'activity_score_stay_active': '활동 유지 점수',
    'activity_score_training_frequency': '운동 빈도 점수',
    'activity_score_training_volume': '운동 점수',
    'activity_steps': '매일 걸음 수',
    'activity_total': '활동 총 시간(분)'
}, inplace=True)
Walk['활동 시작 시간'] = pd.to_datetime(Walk['활동 시작 시간'])
Walk['활동 종료 시간'] = pd.to_datetime(Walk['활동 종료 시간'])
Walk['활동시간'] = Walk['활동 종료 시간'] - Walk['활동 시작 시간']
Walk.drop(['활동 시작 시간', '활동 종료 시간','활동시간'], axis=1, inplace=True)
Walk['하루간 5분당 활동 로그'] = Walk['CONVERT(activity_class_5min USING utf8)'].apply(lambda x: len(x.split('/')))
Walk['하루간 1분 당 MET 로그'] = Walk['CONVERT(activity_met_1min USING utf8)'].apply(lambda x: len(str(x).split('/')))
Walk.drop(['CONVERT(activity_class_5min USING utf8)','CONVERT(activity_met_1min USING utf8)','하루간 1분 당 MET 로그'], axis=1, inplace=True)
Walk['DIAG_NM'] = Walk['DIAG_NM'].replace({'CN': 0, 'MCI': 1, 'Dem': 2})
Walk.drop(["EMAIL"], axis=1, inplace=True)
from sklearn.preprocessing import MinMaxScaler
# Drop the label column 'DIAG_NM' temporarily for normalization
walk_data = Walk.drop(columns=['DIAG_NM'])

# Apply Z-score normalization using StandardScaler
scaler = StandardScaler()
zscore_normalized_data = scaler.fit_transform(walk_data)

# Recreate the Walk_nom dataframe with the normalized data
Walk_nom = pd.DataFrame(zscore_normalized_data, columns=walk_data.columns)

# Add the 'DIAG_NM' label column back to the Walk_nom dataframe
Walk_nom['DIAG_NM'] = Walk['DIAG_NM'].reset_index(drop=True)

# Display the first few rows of the normalized data
Walk_nom


,하루간 평균 MET,하루간 활동 칼로리,하루간 총 사용 칼로리,하루간 5분당 활동 로그,매일 움직인 거리,고강도 활동 시간,비활동 시간,비활동 알람 횟수,저강도 활동 시간,중강도 활동 시간,...,활동 점수,활동 목표달성 점수,매 시간 당 활동유지 점수,회복시간 점수,활동 유지 점수,운동 빈도 점수,운동 점수,매일 걸음 수,활동 총 시간(분),DIAG_NM
0,-0.978203,-1.003840,-0.713049,0.086232,-1.027377,-0.455802,1.448482,-0.579998,-0.717321,-1.064102,...,0.303911,0.076141,0.406237,0.474993,-1.561091,0.576905,0.464069,-1.108327,-0.961115,1
1,-1.139462,-1.191901,-0.970842,0.086232,-1.183968,-0.455802,1.484319,-0.579998,-1.293592,-0.995911,...,0.016229,-0.486413,0.406237,0.474993,-1.306628,0.455965,0.415280,-1.262244,-1.431541,1
2,-1.300720,-1.291462,-1.024082,0.086232,-1.333637,-0.455802,1.405477,0.737933,-1.342987,-1.064102,...,-0.702974,-1.017715,-0.132684,0.474993,-1.497475,-0.299906,-0.218989,-1.390677,-1.494733,1
3,-0.978203,-1.062840,-0.743872,0.086232,-1.132519,-0.455802,1.383974,-0.579998,-0.700856,-1.086833,...,-0.918735,-1.017715,0.406237,0.474993,-1.306628,-0.299906,-1.389946,-1.142814,-0.954093,1
4,-0.010651,-0.347472,0.149998,0.086232,-0.645720,-0.455802,-0.536919,-0.579998,1.834737,-1.064102,...,-0.918735,-1.017715,0.406237,0.474993,0.665467,-1.237187,-2.024214,-0.451040,1.215486,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9700,-0.655686,-0.808405,-0.432839,0.086232,-0.829439,-0.300693,1.921538,4.691725,-0.997224,-0.700413,...,-1.566017,-1.580269,-5.737457,0.474993,-2.133635,-1.237187,0.366490,-0.979214,-1.080477,0
9701,0.311866,0.375271,0.494656,0.086232,0.413939,-0.300693,0.043650,0.737933,0.361129,0.186079,...,-0.343372,-1.017715,-0.132684,0.474993,-0.161541,-0.299906,0.268910,0.194015,0.351867,0
9702,-0.655686,-0.638781,-0.488881,0.086232,-0.693988,0.164635,0.473700,-0.579998,-0.947830,-0.518569,...,-0.918735,-1.580269,0.406237,0.474993,-0.543236,-1.237187,0.366490,-0.846704,-0.961115,0
9703,-0.816944,-1.040715,-0.631788,0.086232,-1.060117,-0.455802,1.907203,0.737933,-1.235965,-0.791335,...,-1.206416,-1.580269,-0.132684,0.474993,-2.006403,-1.237187,0.268910,-1.169486,-1.319200,0


In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# CSV 파일 경로 설정
file_path = '1.Training/원천데이터/2.수면/train_sleep.csv' 
# CSV 파일을 판다스 데이터프레임으로 불러오기
Sleep = pd.read_csv(file_path)

# CSV 파일 경로 설정
file_path = '1.Training/라벨링데이터/2.수면/training_label.csv' 
# CSV 파일을 판다스 데이터프레임으로 불러오기
Sleep_label = pd.read_csv(file_path)


Sleep_label.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
Sleep = pd.merge(Sleep, Sleep_label[['EMAIL', 'DIAG_NM']], on='EMAIL', how='left')
Sleep['EMAIL'] = Sleep['EMAIL'].str.extract(r'(\d{3})')

Sleep = Sleep.drop(['sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min', 'sleep_is_longest',
                    'CONVERT(sleep_hr_5min USING utf8)',
                    'CONVERT(sleep_hypnogram_5min USING utf8)',
                    'CONVERT(sleep_rmssd_5min USING utf8)',
                    'sleep_temperature_deviation',
                    'sleep_temperature_delta'
                    
                    ], axis=1)
Sleep.rename(columns={
    'EMAIL': 'EMAIL',
    'sleep_awake': '깨어있음',
    'sleep_bedtime_end': '종료시간',
    'sleep_bedtime_start': '시작시간',
    'sleep_breath_average': '평균호흡',
    'sleep_deep': '깊은수면',
    'sleep_duration': '총시간',
    'sleep_efficiency': '효율',
    'sleep_hr_average': '평균심박수',
    'sleep_hr_lowest': '최저심박수',
    'sleep_light': '얕은수면',
    'sleep_midpoint_at_delta': '중간점변화',
    'sleep_midpoint_time': '중간점시간',
    'sleep_onset_latency': '잠들기까지시간',
    'sleep_period_id': '기간_ID',
    'sleep_rem': '렘수면',
    'sleep_restless': '뒤척임',
    'sleep_rmssd': '심박변동성_RMSSD',
    'sleep_score': '점수',
    'sleep_score_alignment': '정렬점수',
    'sleep_score_deep': '깊은수면점수',
    'sleep_score_disturbances': '방해점수',
    'sleep_score_efficiency': '효율점수',
    'sleep_score_latency': '잠들기점수',
    'sleep_score_rem': '렘수면점수',
    'sleep_score_total': '총점',
    'sleep_total': '총수면',
}, inplace=True)

# 불필요한 공백이 있을 경우 제거
Sleep.columns = Sleep.columns.str.strip()

Sleep['시작시간'] = pd.to_datetime(Sleep['시작시간'])
Sleep['종료시간'] = pd.to_datetime(Sleep['종료시간'])
Sleep['활동시간'] = Sleep['종료시간'] - Sleep['시작시간']

Sleep.drop(['시작시간', '종료시간','활동시간','EMAIL'], axis=1, inplace=True)
Sleep['DIAG_NM'] = Sleep['DIAG_NM'].replace({'CN': 0, 'MCI': 1, 'Dem': 2})



sleep_data = Sleep.drop(columns=['DIAG_NM'])

# Apply Z-score normalization using StandardScaler
scaler = StandardScaler()
zscore_normalized_sleep = scaler.fit_transform(sleep_data)

# Recreate the Sleep_nom dataframe with the normalized data
Sleep_nom = pd.DataFrame(zscore_normalized_sleep, columns=sleep_data.columns)

# Add the 'DIAG_NM' label column back to the Sleep_nom dataframe
Sleep_nom['DIAG_NM'] = Sleep['DIAG_NM'].reset_index(drop=True)
Sleep_nom

,깨어있음,평균호흡,깊은수면,총시간,효율,평균심박수,최저심박수,얕은수면,중간점변화,중간점시간,...,점수,정렬점수,깊은수면점수,방해점수,효율점수,잠들기점수,렘수면점수,총점,총수면,DIAG_NM
0,0.927613,0.836843,1.909839,1.240800,-0.542695,-1.016011,-0.908284,0.110440,4.465969,0.837267,...,0.935310,-0.225907,0.752378,0.966857,-0.550822,0.633593,0.035711,1.097953,1.022996,1
1,0.310867,0.933395,0.891173,-0.061109,-0.542695,-1.046361,-0.908284,-0.674376,-0.418770,-0.431517,...,0.159106,0.505408,0.671340,0.889506,-0.550822,1.168186,-0.405619,-0.323870,-0.275222,1
2,1.457494,0.933395,-0.184721,0.412313,-1.685799,-1.158106,-1.214628,-0.553635,-0.267022,0.986999,...,-0.444607,0.505408,0.509266,-0.425469,-1.798513,-0.301944,0.443093,-0.470955,-0.384776,1
3,-0.410119,0.547187,0.891173,-0.517622,0.257478,-1.287785,-1.367801,-1.291497,-0.346179,-0.565487,...,0.331596,0.505408,0.671340,1.121560,0.280971,0.232648,0.612835,-0.470955,-0.412164,1
4,0.484598,0.836843,-0.299177,-1.185485,-1.914420,-1.325033,-1.214628,-1.647012,-0.832306,-1.251103,...,-1.824525,-0.619692,0.266154,0.502749,-1.902487,-0.101472,-1.016691,-2.088891,-1.841847,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9700,0.145822,-0.707988,-1.294952,0.598299,0.028857,0.461501,0.317092,1.002581,-0.209935,0.537803,...,0.331596,0.505408,-1.921853,-0.657523,0.021036,0.099000,1.359702,0.705726,0.683378,0
9701,0.058956,-0.514884,-1.947355,-0.196372,-0.314074,0.965041,0.929780,0.291551,-0.081533,-0.147813,...,-0.703342,0.505408,-3.258968,-0.734875,-0.342874,0.433121,1.156011,-0.372899,-0.291655,0
9702,-0.123462,-0.901092,-1.764225,0.894188,0.486099,0.611873,0.776608,1.639825,-0.051074,0.750580,...,0.676575,0.505408,-2.934819,-0.502820,0.540907,0.633593,1.461547,1.245038,1.236627,0
9703,-1.000806,-0.707988,-1.100375,-0.179464,1.286272,0.916757,0.776608,0.680605,-0.421019,-0.155694,...,0.504086,0.505408,-1.516667,-0.502820,1.164752,0.633593,1.054165,0.411555,0.398537,0


In [52]:
merged_df = pd.concat([Walk_nom, Sleep_nom], axis=1)

# 중복된  열 삭제 (하나는 유지)
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]


In [55]:
# Rename the 'DIAG_NM' column to 'label'
merged_df.rename(columns={'DIAG_NM': 'label'}, inplace=True)
# Extract the 'label' column
train_label = merged_df[['label']]
# Save the 'label' column to a CSV file
train_label.to_csv('train_label.csv', index=False)

# # Save the rest of the dataframe (excluding 'label') to another CSV file
ver5_df = merged_df.drop(columns=['label'])
ver5_df.to_csv('train_ver5.csv', index=False)